# Running on Slicer 5.6 Jupiter extension

In [2]:
import JupyterNotebooksLib as slicernb
import slicer
import vtk
import base64
import pandas as pd

In [3]:
def reset_scene():
  """
  Resets the Slicer scene by deleting all nodes and sets default view settings.
  """
  
      # Set viewer size to 50% of screen size
  slicernb.AppWindow.setWindowSize(scale=1)
    # Hide patient information from slice view
  slicernb.showSliceViewAnnotations(True)
    # Hide slice view controllers
  slicer.mrmlScene.Clear(False)



In [4]:
def load_data(case_number, dataset_path, case_names):
  """
  Loads CT and MR volumes from a specific case

  Args:
      case_number: Integer representing the case number (1-42)
      dataset_path: String containing the path to the dataset directory
      case_names: List of strings containing case names

  Returns:
      A tuple containing the loaded CT and MR volumes as slicer nodes
  """
  CT_volume = slicer.util.loadVolume(dataset_path + f"\\{case_names[case_number]}\\{case_names[case_number]}_IMG_CT.nrrd")
  MR_volume = slicer.util.loadVolume(dataset_path + f"\\{case_names[case_number]}\\{case_names[case_number]}_IMG_MR_T1.nrrd")
  CT_volume.AddCenteringTransform()
  MR_volume.AddCenteringTransform()
  slicer.vtkSlicerTransformLogic().hardenTransform(CT_volume)
  slicer.vtkSlicerTransformLogic().hardenTransform(MR_volume)
  
  return CT_volume, MR_volume

In [5]:
def create_color_table(colorTableNode, COLOR_dict):
  """
  Sets colors in a color table node based on the provided color dictionary

  Args:
      color_table_node: slicer mrml color table node
      COLOR_dict: Dictionary mapping segment names to RGB color tuples
  """
  colorTableNode.SetTypeToUser()
  colorTableNode.SetName("HaN-Seg")
  colorTableNode.SetNumberOfColors(len(COLOR_dict))
  colorTableNode.HideFromEditorsOff()
  slicer.mrmlScene.AddNode(colorTableNode); colorTableNode.UnRegister(None)
  colorTableNode.SetNamesInitialised(True)
  for segment_name, color in COLOR_dict.items():
      colorTableNode.AddColor(segment_name, color[0]/255, color[1]/255, color[2]/255, 1.0)         
          

In [6]:
def load_and_color_segmentation(case_number, dataset_path, case_names, LABEL_dict, color_table_node, COLOR_dict):
  """
  Loads segmentations for all segments except background, assigns colors,
  and adds them to the stacked segments node

  Args:
      case_number: Integer representing the case number (1-42)
      dataset_path: String containing the path to the dataset directory
      case_names: List of strings containing case names
      LABEL_dict: Dictionary mapping segment names to integer IDs
      color_table_node: slicer mrml color table node
  """
  print(f"{case_number + 1} : Loading segmentations...")
  stacked_segments = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLSegmentationNode', 'stacked_segments')
  stacked_seg = stacked_segments.GetSegmentation()
  list_of_segments = list(LABEL_dict.keys())
  list_of_segments.remove('background')
  
  for seg_name in list_of_segments:
      segmentation = slicer.util.loadSegmentation(
          dataset_path + f"\\{case_names[case_number]}\\{case_names[case_number]}_OAR_{seg_name}.seg.nrrd",
          properties={
              'name': seg_name,
              'colorNodeID': color_table_node.GetID()
          },
          returnNode=True
      )
      segmentationNode = slicer.util.getNode(seg_name)
      segmentation = segmentationNode.GetSegmentation()
  
      # Get segment ID from LABEL_dict
      segmentID = 'A_Carotid_L'
  
      # Set segment name and color directly
      segmentation.GetSegment(segmentID).SetName(seg_name)
      segmentColor = COLOR_dict[seg_name]
      segmentation.GetSegment(segmentID).SetColor(segmentColor[0]/255, segmentColor[1]/255, segmentColor[2]/255)
  
      stacked_seg.CopySegmentFromSegmentation(segmentation, segmentID)  # Copy using correct segment ID
  return stacked_segments
  

In [35]:
def center_and_transform_segmentation(stacked_segments):
  """
  Calculates the center of the segmentation and applies a transform to move it there

  Args:
    stacked_segments: slicer mrml segmentation node
  """
  bounds = [0]*6
  stacked_segments.GetRASBounds(bounds)
  center = [(bounds[1]+bounds[0])/2, (bounds[3]+bounds[2])/2, (bounds[5]+bounds[4])/2]

  transformNode = slicer.vtkMRMLLinearTransformNode()
  slicer.mrmlScene.AddNode(transformNode)

  matrix = vtk.vtkMatrix4x4()
  matrix.SetElement(0, 3, -center[0])
  matrix.SetElement(1, 3, -center[1])
  matrix.SetElement(2, 3, -center[2])
  transformNode.SetMatrixTransformToParent(matrix)

  stacked_segments.SetAndObserveTransformNodeID(transformNode.GetID())
  slicer.vtkSlicerTransformLogic().hardenTransform(stacked_segments)


In [8]:
def visualize_and_save_data(case_number, dataset_path, case_names, output_dir, CT_volume, MR_volume, stacked_segments):
  """
  Visualizes data in slice views and saves volumes and segmentation

  Args:
      case_number: Integer representing the case number (1-42)
      dataset_path: String containing the path to the dataset directory
      case_names: List of strings containing case names
      output_dir: String containing the path to the output directory
      CT_volume: slicer mrml volume node for CT data
      MR_volume: slicer mrml volume node for MR data
      stacked_segments: slicer mrml segmentation node for combined segments
  """
  slicernb.showVolumeRendering(CT_volume, False, presetName=None)
  slicernb.showVolumeRendering(MR_volume, False, presetName=None)
  stacked_segments.CreateClosedSurfaceRepresentation()

  slicer.util.setSliceViewerLayers(background=CT_volume)
  slicer.util.setSliceViewerLayers(foreground=MR_volume)
  slicer.util.setSliceViewerLayers(foregroundOpacity=0.7)
  slicer.util.setSliceViewerLayers(label=stacked_segments)

  displayNode = stacked_segments.GetDisplayNode()
  if displayNode is not None:
      displayNode.SetVisibility(1)
      displayNode.SetOpacity(1.0)

  result = slicernb.ViewDisplay('FourUp', center=True)
  print(result)
  print(f"{case_number + 1} : Saving data...")
  # Save CT, MR, and segmentation volumes
  slicer.util.saveNode(CT_volume, output_dir + f"\\{case_names[case_number]}\\{case_names[case_number]}_IMG_CT.nrrd")
  slicer.util.saveNode(MR_volume, output_dir + f"\\{case_names[case_number]}\\{case_names[case_number]}_IMG_MR_T1.nrrd")
  slicer.util.saveNode(stacked_segments, output_dir + f"\\{case_names[case_number]}\\{case_names[case_number]}_stacked_segments.seg.nrrd")

  # Save screenshot as PNG
  image_data = base64.b64decode(result.dataValue)
  with open(output_dir + f"\\{case_names[case_number]}\\{case_names[case_number]}_result.png", 'wb') as file:
      file.write(image_data)

In [38]:
def main():
    """
    The main function that processes all cases
    """
    dataset_path = "C:\\Users\\benam\Downloads\HaN-Seg\HaN-Seg\set_1"
    case_names = [f"case_{num:02d}" for num in range(1, 43)]
    output_dir = "C:\\Users\\benam\Downloads\HaN-Seg\HaN-Seg\set_2"
    df = pd.read_csv(dataset_path + '\\OAR_data.csv', delimiter=';')
    # Iterate over the DataFrame rows
    missing_data = []
    for index, row in df.iterrows():
        # Get the case name
        case_name = row.name
        # Find the segments that are missing (value is 0)
        missing_segments = row[row == 0].index.tolist()
        # Print the case name and the missing segments
        if(len(missing_segments) > 0):
            print(f"For {case_name}, the missing segments are: {missing_segments}")
            missing_data.append((case_name, missing_segments))

    missing_data_dict = {case_num: segments for case_num, segments in missing_data}

    for case_number in range(len(case_names)):
        LABEL_dict = {
            "background": 0,
            "A_Carotid_L": 1,
            "A_Carotid_R": 2,
            "Arytenoid": 3,
            "Bone_Mandible": 4,
            "Brainstem": 5,
            "BuccalMucosa": 6,
            "Cavity_Oral": 7,
            "Cochlea_L": 8,
            "Cochlea_R": 9,
            "Cricopharyngeus": 10,
            "Esophagus_S": 11,
            "Eye_AL": 12,
            "Eye_AR": 13,
            "Eye_PL": 14,
            "Eye_PR": 15,
            "Glnd_Lacrimal_L": 16,
            "Glnd_Lacrimal_R": 17,
            "Glnd_Submand_L": 18,
            "Glnd_Submand_R": 19,
            "Glnd_Thyroid": 20,
            "Glottis": 21,
            "Larynx_SG": 22,
            "Lips": 23,
            "OpticChiasm": 24,
            "OpticNrv_L": 25,
            "OpticNrv_R": 26,
            "Parotid_L": 27,
            "Parotid_R": 28,
            "Pituitary": 29,
            "SpinalCord": 30,
        }
        colors = [
            [244, 214, 49],    # SpinalCord
            [216, 101, 79],    # A_Carotid_L
            [216, 101, 79],    # A_Carotid_R
            [183, 156, 220],   # Arytenoid
            [222, 198, 101],   # Bone_Mandible
            [145, 92, 109],    # Brainstem
            [178, 69, 182],    # BuccalMucosa
            [121, 39, 153],    # Cavity_Oral
            [104, 181, 63],    # Cochlea_L
            [123, 174, 91],    # Cochlea_R
            [220, 127, 211],   # Cricopharyngeus
            [174, 125, 64],    # Esophagus_S
            [127, 75, 38],     # Eye_AL
            [127, 75, 38],     # Eye_AR
            [53, 152, 174],    # Eye_PL
            [53, 152, 174],    # Eye_PR
            [86, 58, 127],     # Glnd_Lacrimal_L
            [86, 58, 127],     # Glnd_Lacrimal_R
            [222, 198, 101],   # Glnd_Submand_L
            [222, 198, 101],   # Glnd_Submand_R
            [62, 162, 114],    # Glnd_Thyroid
            [47, 210, 120],    # Glottis
            [150, 208, 243],   # Larynx_SG
            [188, 91, 95],     # Lips
            [99, 106, 24],     # OpticChiasm
            [127, 24, 70],     # OpticNrv_L
            [127, 24, 70],     # OpticNrv_R
            [31, 45, 172],     # Parotid_L
            [31, 45, 172],     # Parotid_R
            [57, 157, 110]     # Pituitary
        ]
        COLOR_dict = {
            "SpinalCord": colors[0],  # Red
            "A_Carotid_L": colors[1],  # Green
            "A_Carotid_R": colors[2],  # Blue
            "Arytenoid": colors[3],    # Yellow
            "Bone_Mandible": colors[4], # Magenta
            "Brainstem": colors[5],    # Cyan
            "BuccalMucosa": colors[6],  # Maroon
            "Cavity_Oral": colors[7],   # Green (Lime)
            "Cochlea_L": colors[8],     # Navy
            "Cochlea_R": colors[9],     # Purple
            "Cricopharyngeus": colors[10], # Olive
            "Esophagus_S": colors[11],   # Teal
            "Eye_AL": colors[12],       # Orange
            "Eye_AR": colors[13],       # Spring Green
            "Eye_PL": colors[14],       # Indigo
            "Eye_PR": colors[15],       # Gold
            "Glnd_Lacrimal_L": colors[16], # Deep Pink
            "Glnd_Lacrimal_R": colors[17], # Black
            "Glnd_Submand_L": colors[18], # White
            "Glnd_Submand_R": colors[19], # Gray
            "Glnd_Thyroid": colors[20],  # Dark Orange
            "Glottis": colors[21],      # Sky Blue
            "Larynx_SG": colors[22],    # Dark Cyan
            "Lips": colors[23],         # Light Pink
            "OpticChiasm": colors[24],   # Dark Red
            "OpticNrv_L": colors[25],    # Dark Blue
            "OpticNrv_R": colors[26],    # Dark Green
            "Parotid_L": colors[27],     # Dark Purple
            "Parotid_R": colors[28],     # Light Gold
            "Pituitary": colors[29],    # Light Sky Blue
        }
        try:
            removed_colors=[]
            if case_number in missing_data_dict:
                removed_colors.append(case_number) 
                removed_labels = missing_data_dict[case_number]
                for color in removed_colors:
                    colors.remove(colors[color])
                for color in removed_colors:
                    COLOR_dict.pop(color, None)
                for label in removed_labels:
                    LABEL_dict.pop(label, None)

            reset_scene()
            print(f"Processing case {case_number + 1}/{len(case_names)}: {case_names[case_number]}")

            # Load CT and MR volumes
            CT_volume, MR_volume = load_data(case_number, dataset_path, case_names)

            # Create color table
            colorTableNode = slicer.mrmlScene.CreateNodeByClass("vtkMRMLColorTableNode")
            create_color_table(colorTableNode, COLOR_dict)

            # Load and color segmentation
            stacked_segments = load_and_color_segmentation(case_number, dataset_path, case_names, LABEL_dict, colorTableNode, COLOR_dict)

            # Center segmentation
            center_and_transform_segmentation(stacked_segments)

            # Visualize and save data
            visualize_and_save_data(case_number, dataset_path, case_names, output_dir, CT_volume, MR_volume, stacked_segments)
           
        except Exception as e:
            print(f"Error processing case {case_number}: {e}")

if __name__ == "__main__":
    main()

For 18, the missing segments are: ['OpticChiasm']
[(18, ['OpticChiasm'])]
Processing case 1/42: case_01
1 : Loading segmentations...
1 : Saving data...
Processing case 2/42: case_02
2 : Loading segmentations...
2 : Saving data...
Processing case 3/42: case_03
3 : Loading segmentations...
3 : Saving data...
Processing case 4/42: case_04
4 : Loading segmentations...
4 : Saving data...
Processing case 5/42: case_05
5 : Loading segmentations...
5 : Saving data...
Processing case 6/42: case_06
6 : Loading segmentations...
6 : Saving data...
Processing case 7/42: case_07
7 : Loading segmentations...
7 : Saving data...
Processing case 8/42: case_08
8 : Loading segmentations...
8 : Saving data...
Processing case 9/42: case_09
9 : Loading segmentations...
9 : Saving data...
Processing case 10/42: case_10
10 : Loading segmentations...
10 : Saving data...
Processing case 11/42: case_11
11 : Loading segmentations...
11 : Saving data...
Processing case 12/42: case_12
12 : Loading segmentations...
1

# Conclusion
# In this notebook, we have demonstrated how we used 3D Slicer's Jupiter Kernel extension to automatically process our medical data and apply the necessary transforms to clean it, and finally end up with a new set saved as a new clean dataset.